In [180]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from tqdm import tqdm 


In [215]:
def standard(train) :
    
    train['HomePlanet'] = train['HomePlanet'].fillna('Earth')
    
    train['CryoSleep'] = train['CryoSleep'].fillna(False)
        
    train['Destination'] = train['Destination'].fillna('TRAPPIST-1e')
    
    train['Age'] = train['Age'].fillna(train['Age'].mean())
    
    train['VIP'] = train['VIP'].fillna(False)
    
    train['RoomService'] = train['RoomService'].fillna(0.0)
    
    train['FoodCourt'] = train['FoodCourt'].fillna(0.0)
    
    train['ShoppingMall'] = train['ShoppingMall'].fillna(0.0)
    
    train['Spa'] = train['Spa'].fillna(0.0)
    
    train['VRDeck'] = train['VRDeck'].fillna(0.0)
    
    
    ##################################
    #cabin
    #Deck Side
    train['Cabin'] = train['Cabin'].fillna('-------')
    Deck = []
    Side = []
    time = 0
    for i in train['Cabin'] :
        if i[0] == '-' :
            if time % 2 == 0 :
                time += 1
                Deck.append('F')
                Side.append('P')
            else :
                Deck.append('G')
                Side.append('S')
        else :
            Deck.append(i[0])
            Side.append(i[-1])

    train['Deck'] = Deck
    train['Side'] = Side
    
    
    
    
    ##num
    Num = []
    for i in train['Cabin'] :
        if '-' in i[2:-2] :
            Num.append('600')
        else :
            Num.append(i[2:-2])

    train['Num'] = Num
    ######################################
    
    
    
    #one hot encoding
    x = train[['HomePlanet', 'CryoSleep', 'Destination', 'Age',
        'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
        'Deck', 'Side']]

    x = pd.get_dummies(x)

    x['Num'] = train['Num']
    
    
    #標準化
    scaler = StandardScaler()
    x[['Age','RoomService','FoodCourt',
       'ShoppingMall','Spa','VRDeck']] = scaler.fit_transform(x[['Age','RoomService','FoodCourt',
                                            'ShoppingMall','Spa','VRDeck']])
    
    return x

In [216]:
train = pd.read_csv("C:/Users/asd11/OneDrive/桌面/python專案/space titanic/train.csv")
test = pd.read_csv("C:/Users/asd11/OneDrive/桌面/python專案/space titanic/test.csv")
test_id = test['PassengerId']
ans = train['Transported']

In [217]:
train = standard(train)
test = standard(test)

In [218]:
train_x,test_x,train_y,test_y = train_test_split(train,ans,test_size=0.2,random_state=10)

In [219]:
depth = []
for i in tqdm(range(2,50,1)) :
    model = RandomForestClassifier(max_depth=i)
    model.fit(train_x,train_y)
    scores = cross_val_score(model, test_x, test_y, cv=5)
    depth.append(scores.mean())
final_depth = np.argmax(np.array(depth)) + 2
print(final_depth)
estimators = []
for i in tqdm(range(2,50,1)) :
    model = RandomForestClassifier(max_depth=7,n_estimators =i)
    model.fit(train_x,train_y)
    scores = cross_val_score(model, test_x, test_y, cv=5)
    estimators.append(scores.mean())
final_estimators = np.argmax(np.array(estimators)) + 2
print(final_estimators)

  4%|███▍                                                                               | 2/48 [00:00<00:02, 15.47it/s]

8


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:13<00:00,  3.45it/s]

48


In [220]:
model = RandomForestClassifier(max_depth=final_depth,n_estimators=final_estimators)
model.fit(train_x,train_y)
pred = model.predict(test)
submission = pd.DataFrame({'PassengerId':test_id,'Transported':pred})
submission.to_csv('submission.csv',index=False)